In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler
import streamlit as st

In [2]:
'''
Model Test
''' 



# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = "./EXAONE-3.5-2.4B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map='auto',
#     trust_remote_code=True
# )
# instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

# messages = [
#     {"role": "user", "content": f"{instruction}"}
#     ]

# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# terminators = [
#     tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9
# )

# print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

'\nModel Test\n'

In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import bitsandbytes as bnb
from trl import SFTTrainer

c:\ProgramData\Anaconda3\envs\llm_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
# Installing More Dependencies
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer, SFTConfig
import os

In [35]:
model_id = './models/EEVE-Korean-Instruct-10.8B-v1.0'

In [4]:
def get_model_and_tokenizer(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
  )
  model = AutoModelForCausalLM.from_pretrained(
      model_id, 
      quantization_config=bnb_config, 
      device_map='auto',
      trust_remote_code = True,
  )

  model.config.use_cache=False
  model.config.pretraining_tp=1
  return model, tokenizer

In [5]:
model, tokenizer = get_model_and_tokenizer(model_id)

Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.28it/s]


In [6]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [20]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=1024,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [21]:
output_model="finetuningTest"

In [22]:
def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [23]:
training_data1 = [{"prompt": "체계개발실 지능팀 구성원에 대해 알려줘", "response": "신동헌 팀장님, 김태훈 중위님, 김남현 중위님, 천용 중사님, 신성환 중사님, 최훈진 주무관님, 최준호 병장, 김영주 일병으로 구성되어있습니다."}]

In [24]:
import pandas as pd
def prepare_train_datav2(data):
    # Convert the data to a Pandas DataFrame
    data_df = pd.DataFrame(data)
    # Create a new column called "text"
    data_df["text"] = data_df[["prompt", "response"]].apply(lambda x: "<|im_start|>user\n" + x["prompt"] + " <|im_end|>\n<|im_start|>assistant\n" + x["response"] + "<|im_end|>\n", axis=1)
    # Create a new Dataset from the DataFrame
    data = Dataset.from_pandas(data_df)
    return data

In [25]:
data = prepare_train_datav2(training_data1)

In [26]:
peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules = [
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
        ],
    )

In [27]:
arg = SFTConfig(
        output_dir=output_model,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=100,
        # max_steps=,
        fp16=True,
        packing = False,
        max_seq_length = 1024
    )

In [28]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        args=arg,
        tokenizer=tokenizer,
    )

C:\Users\AFOC\AppData\Local\Temp\ipykernel_21836\1023881791.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Map: 100%|██████████| 1/1 [00:00<00:00, 501.83 examples/s]


In [16]:
trainer.train()

 10%|█         | 10/100 [00:23<03:18,  2.21s/it]

{'loss': 3.7342, 'grad_norm': 3.2032907009124756, 'learning_rate': 0.0001968583161128631, 'epoch': 10.0}


KeyboardInterrupt: 

In [33]:
generate_response(user_input = "경기도 수원시에서 아이들과 갈 수 있는 관광 명소 1곳을 알려줘")

<|im_start|>user
경기도 수원시에서 아이들과 갈 수 있는 관광 명소 1곳을 알려줘<|im_end|>
<|im_start|>assistant: 경기도 수원시에는 가족 단위로 즐길 수 있는 다양한 관광지가 있습니다. 그 중 하나로 **수원 화성**을 추천드립니다!

### 수원 화성 (華城)
- **위치**: 경기 수원시 팔달구 화서동
- **특징**:
  - 조선 시대에 건설된 성곽으로, 유네스코 세계문화유산으로 등재되어 있어요.'/>'
    + 아름다운 건축물들이 가득하고 역사적 의미도 깊어서 교육적으로 매우 가치있죠。'/></li><ul class="list"><div markdown="1">• **역사 탐방** : 성벽을 따라 걷다 보면 과거의 모습이 생생하게 떠오르고, 다양한 유적 및 전시관 방문 가능</span>><br/><p style="">• **가족 활동 공간** </strong>>><!-- -->* **연등축제나 수원화성국제음악회 등 문화 행사 참여가능합니다.</style-->'</stylediv--><---></body->
Time taken for inference: 8.43 seconds
